## Import libraries

In [30]:
import cv2
import os
import shutil
import mediapipe as mp
import numpy as np # Za np.zeros_like, če bi bil potrebno

In [31]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
# Ali še bolj specifično za Metal:
print(tf.config.list_physical_devices('GPU'))

Num GPUs Available:  1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [32]:
print(f"MediaPipe verzija: {mp.__version__}")
print(f"OpenCV verzija: {cv2.__version__}")

MediaPipe verzija: 0.10.11
OpenCV verzija: 4.11.0


In [33]:
# Model_selection=1 je za zaznavanje obrazov na razdalji (celotni obraz), bolj robustno
# min_detection_confidence=0.7 pomeni, da je potrebna visoka zanesljivost za zaznavo
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.7)

I0000 00:00:1748522180.564328 3962523 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4 Pro


In [34]:
# --- Parametri ---
TARGET_IMG_SIZE = (224, 224) # Željena velikost izrezanih obrazov
MARGIN_FACTOR = 1.5         # Faktor za razširitev izrezanega okvirja obraza (1.0 = samo obraz)

In [35]:
# Pot do mape, kjer so shranjene podmape z LFW slikami (npr. 'Aaron_Eckhart/', 'Abel_Pacheco/')
# PREVERI TO POT NA SVOJEM SISTEMU!
# Glede na tvoj tree diagram, je verjetno to:
INPUT_LFW_RAW_FOLDER = '../data/raw/Ifw-dataset/lfw-deepfunneled/lfw-deepfunneled'

# Pot do mape, kamor bomo shranili obdelane "false data" obraze
OUTPUT_PROCESSED_FALSE_DATA_FOLDER = '../data/processed/false_data'

In [36]:
# --- Funkcija za izrezovanje in prilagoditev obraza ---
def crop_face_only(image, detection, target_size=TARGET_IMG_SIZE, margin_factor=MARGIN_FACTOR):
    h, w, _ = image.shape
    location_data = detection.location_data
    relative_bounding_box = location_data.relative_bounding_box

    # Izračun absolutnih koordinat okvirja
    xmin = int(relative_bounding_box.xmin * w)
    ymin = int(relative_bounding_box.ymin * h)
    box_width = int(relative_bounding_box.width * w)
    box_height = int(relative_bounding_box.height * h)

    xmax = xmin + box_width
    ymax = ymin + box_height

    # Izračun centra okvirja
    center_x = (xmin + xmax) / 2
    center_y = (ymin + ymax) / 2

    # Določitev največje dimenzije in dodajanje marže
    max_dim = max(box_width, box_height) * margin_factor
    
    # Izračun novih koordinat izrezanega okvirja z maržo
    x1 = int(center_x - max_dim / 2)
    y1 = int(center_y - max_dim / 2)
    x2 = int(center_x + max_dim / 2)
    y2 = int(center_y + max_dim / 2)

    # Poskrbimo, da okvir ne preseže meja slike
    x1 = max(0, x1)
    y1 = max(0, y1)
    x2 = min(w, x2)
    y2 = min(h, y2)

    # Izrezovanje obraza
    cropped_face = image[y1:y2, x1:x2]
    
    # Preverimo, če je izrezan obraz prazen (lahko se zgodi, če so koordinate napačne)
    if cropped_face.shape[0] == 0 or cropped_face.shape[1] == 0:
        return None

    # Sprememba velikosti izrezanega obraza na ciljno velikost
    final_face = cv2.resize(cropped_face, target_size, interpolation=cv2.INTER_AREA)
    return final_face

In [37]:
# --- Priprava izhodne mape ---
# Preverimo, ali izhodna mapa že obstaja, in jo po potrebi izbrišemo/ustvarimo
if os.path.exists(OUTPUT_PROCESSED_FALSE_DATA_FOLDER):
    print(f"Izhodna mapa '{OUTPUT_PROCESSED_FALSE_DATA_FOLDER}' že obstaja. Brišem jo in ustvarjam novo.")
    shutil.rmtree(OUTPUT_PROCESSED_FALSE_DATA_FOLDER)
os.makedirs(OUTPUT_PROCESSED_FALSE_DATA_FOLDER, exist_ok=True)

# --- Glavni proces obdelave LFW slik ---
total_processed_faces = 0
total_images_skipped = 0

print(f"Začenjam z obdelavo LFW slik iz: {INPUT_LFW_RAW_FOLDER}")

Začenjam z obdelavo LFW slik iz: ../data/raw/Ifw-dataset/lfw-deepfunneled/lfw-deepfunneled


In [38]:
# LFW ima podmape za vsako osebo (npr. 'Aaron_Eckhart/', 'Abel_Pacheco/')
# Iteriramo skozi te podmape in nato skozi slike v vsaki podmapi
for person_folder_name in os.listdir(INPUT_LFW_RAW_FOLDER):
    person_folder_path = os.path.join(INPUT_LFW_RAW_FOLDER, person_folder_name)

    # Preverimo, če je to res mapa
    if os.path.isdir(person_folder_path):
        for image_name in os.listdir(person_folder_path):
            # Preverimo, če je to slika (če ima ustrezno končnico)
            if image_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(person_folder_path, image_name)
                
                # Naloži sliko (OpenCV bere v BGR formatu)
                image_bgr = cv2.imread(image_path)

                if image_bgr is None:
                    print(f"Opozorilo: Slike ni mogoče naložiti: {image_path}. Preskakujem.")
                    total_images_skipped += 1
                    continue

                # Pretvori BGR v RGB, saj MediaPipe deluje z RGB
                image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

                # Zaznavanje obraza z MediaPipe
                results = face_detection.process(image_rgb)

                if results.detections:
                    # MediaPipe lahko zazna več obrazov. Za LFW se ponavadi predpostavlja 1 obraz na sliko.
                    # Če jih je več, obdelamo samo prvega zaznanega.
                    processed_face_rgb = crop_face_only(image_rgb, results.detections[0], target_size=TARGET_IMG_SIZE)
                    
                    if processed_face_rgb is None:
                        print(f"Opozorilo: Obraz v sliki {image_name} (oseba: {person_folder_name}) ni bil pravilno izrezan. Preskakujem.")
                        total_images_skipped += 1
                        continue

                    # Pretvori obdelan obraz nazaj v BGR za shranjevanje z OpenCV
                    processed_face_bgr = cv2.cvtColor(processed_face_rgb, cv2.COLOR_RGB2BGR)

                    # Ustvari unikatno ime datoteke za shranjevanje
                    # Uporabimo ime osebe in originalno ime slike za edinstvenost
                    unique_filename = f"{person_folder_name}_{os.path.splitext(image_name)[0]}.jpg"
                    output_path = os.path.join(OUTPUT_PROCESSED_FALSE_DATA_FOLDER, unique_filename)
                    
                    cv2.imwrite(output_path, processed_face_bgr)
                    total_processed_faces += 1
                else:
                    print(f"Opozorilo: Obraz ni zaznan v sliki: {image_path}. Preskakujem.")
                    total_images_skipped += 1

Opozorilo: Obraz ni zaznan v sliki: ../data/raw/Ifw-dataset/lfw-deepfunneled/lfw-deepfunneled/Mohammad_Fares/Mohammad_Fares_0001.jpg. Preskakujem.
Opozorilo: Obraz ni zaznan v sliki: ../data/raw/Ifw-dataset/lfw-deepfunneled/lfw-deepfunneled/Ziwang_Xu/Ziwang_Xu_0001.jpg. Preskakujem.
Opozorilo: Obraz ni zaznan v sliki: ../data/raw/Ifw-dataset/lfw-deepfunneled/lfw-deepfunneled/David_Wells/David_Wells_0003.jpg. Preskakujem.
Opozorilo: Obraz ni zaznan v sliki: ../data/raw/Ifw-dataset/lfw-deepfunneled/lfw-deepfunneled/Joseph_Biden/Joseph_Biden_0001.jpg. Preskakujem.
Opozorilo: Obraz ni zaznan v sliki: ../data/raw/Ifw-dataset/lfw-deepfunneled/lfw-deepfunneled/Saoud_Al_Faisal/Saoud_Al_Faisal_0001.jpg. Preskakujem.
Opozorilo: Obraz ni zaznan v sliki: ../data/raw/Ifw-dataset/lfw-deepfunneled/lfw-deepfunneled/Sebastian_Saja/Sebastian_Saja_0003.jpg. Preskakujem.
Opozorilo: Obraz ni zaznan v sliki: ../data/raw/Ifw-dataset/lfw-deepfunneled/lfw-deepfunneled/Fazal-ur-Rehman/Fazal-ur-Rehman_0001.jpg. 

In [39]:
print(f"\n--- Postopek obdelave LFW zaključen ---")
print(f"Skupno obdelanih in shranjenih obrazov: {total_processed_faces}")
print(f"Skupno preskočenih slik (brez zaznanega obraza ali napake pri obdelavi): {total_images_skipped}")

# Zapri MediaPipe sejo
face_detection.close()


--- Postopek obdelave LFW zaključen ---
Skupno obdelanih in shranjenih obrazov: 12947
Skupno preskočenih slik (brez zaznanega obraza ali napake pri obdelavi): 286
